In [5]:
with open("src/grammar/interp.lex") as infile:
    flines = [line.strip() for line in infile if line.strip() != '']

In [3]:
def get_char_set(set_str: str) -> set[str]:
    chset: set[str] = set()

    i = 0
    prev = ''
    while i < len(set_str):
        if prev and set_str[i] == '-' and i+1 < len(set_str):
            start, end = ord(prev), ord(set_str[i+1])
            for j in range(start, end+1):
                chset.add(chr(j))
            i += 1
            prev = ''
        else:
            chset.add(set_str[i])
            prev = set_str[i]
        i += 1
    
    return chset

In [6]:
delta: dict[str, dict[str, tuple[str, str, bool]]] = {}
rsvd: dict[str, str] = {}
in_rsv = False

for line in flines:
    if 'RESERVED' in line:
        in_rsv = True
        continue

    if in_rsv:
        word, _, tok = line.split()
        rsvd[word] = tok
    else:
        consume = True
        nxt_state = ''
        ret_val = ''

        if line[0] == '[':
            consume = False
            line = line[1:-1]
        
        s, _, ch_str, t_or_ret = line.split()
        if s not in delta: delta[s] = {}

        chset: set[str] = set()
        if ch_str[0] == '[':
            chset = get_char_set(ch_str[1:-1])
        elif ch_str == 'WS': chset = { ' ', '\t', '\n' }
        elif ch_str == 'NEWLINE': chset = { '\n' }
        elif ch_str == 'TAB': chset = { '\t' }
        elif ch_str == 'SPACE': chset = { ' ' }
        else: chset = { ch_str }

        if t_or_ret[0] == '<':
            ret_val = t_or_ret[1:-1]
        else:
            nxt_state = t_or_ret
        
        for c in chset:
            delta[s][c] = (nxt_state, ret_val, consume)

In [8]:
delta

{'tok': {'\n': ('tok', '', False),
  '\t': ('tok', '', False),
  ' ': ('tok', '', False),
  '(': ('', 'LPAREN', True),
  ')': ('', 'RPAREN', True),
  '+': ('', 'PLUS', True),
  '-': ('', 'MINUS', True),
  '/': ('', 'SLASH', True),
  '%': ('', 'MODULO', True),
  '*': ('star', '', True),
  '0': ('num0', '', True),
  '6': ('num1', '', True),
  '9': ('num1', '', True),
  '4': ('num1', '', True),
  '2': ('num1', '', True),
  '8': ('num1', '', True),
  '7': ('num1', '', True),
  '3': ('num1', '', True),
  '5': ('num1', '', True),
  '1': ('num1', '', True),
  '.': ('float', '', True),
  'k': ('ident', '', True),
  't': ('ident', '', True),
  'h': ('ident', '', True),
  'l': ('ident', '', True),
  'I': ('ident', '', True),
  'O': ('ident', '', True),
  '_': ('ident', '', True),
  'X': ('ident', '', True),
  'V': ('ident', '', True),
  'n': ('ident', '', True),
  'q': ('ident', '', True),
  'x': ('ident', '', True),
  'Y': ('ident', '', True),
  'b': ('ident', '', True),
  'd': ('ident', '', Tr